In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import warnings
from sklearn.model_selection import train_test_split
import importlib 
import warnings
import urllib3
warnings.filterwarnings('ignore')
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Suppress TensorFlow logging

## 1. Create Directories and Load Data

In [ ]:
import config
importlib.reload(config)

from config import DatasetConfig

# Create necessary directories
os.makedirs(DatasetConfig.PATHS['results'], exist_ok=True)
os.makedirs('model', exist_ok=True)

# Load the annotations
print("Loading annotations...")
annotations = pd.read_csv(DatasetConfig.PATHS['csv'])
print("Annotations shape:", annotations.shape)

## 2. Analyze Dataset

In [ ]:
import data_analyzer
importlib.reload(data_analyzer)

from data_analyzer import DataAnalyzer

analyzer = DataAnalyzer(annotations, DatasetConfig.PATHS['results'])
analyzer.analyze_data()

## 3. Split Dataset and Create Data Generators

In [ ]:
import data_generator
importlib.reload(data_generator)

from data_generator import GlomeruliDataGenerator

train_val_df, test_df = train_test_split(annotations, test_size=0.2, random_state=42)
train_df, val_df = train_test_split(train_val_df, test_size=0.25, random_state=42)

print("Dataset splits:")
print(f"Training samples: {len(train_df)}")
print(f"Validation samples: {len(val_df)}")
print(f"Test samples: {len(test_df)}")

In [ ]:
# Create data generators for each set
train_generator = GlomeruliDataGenerator(
    train_df, 
    DatasetConfig.PATHS['base'],
    batch_size=DatasetConfig.TRAINING['batch_size'],
    image_size=DatasetConfig.TRAINING['image_size']
)

val_generator = GlomeruliDataGenerator(
    val_df,
    DatasetConfig.PATHS['base'],
    batch_size=DatasetConfig.TRAINING['batch_size'],
    image_size=DatasetConfig.TRAINING['image_size'],
    shuffle=False
)

test_generator = GlomeruliDataGenerator(
    test_df,
    DatasetConfig.PATHS['base'],
    batch_size=DatasetConfig.TRAINING['batch_size'],
    image_size=DatasetConfig.TRAINING['image_size'],
    shuffle=False
)

print("Data generators created successfully")

## 4. ResNet

In [ ]:
import resnet.resnet_model_builder as resnet_model_builder
importlib.reload(resnet_model_builder)

from resnet.resnet_model_builder import ModelBuilder

# Create model with input shape
model = ModelBuilder.create_model(input_shape=(224, 224, 3))

# Print model summary to verify architecture
model.summary()

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import resnet.resnet_model_trainer as resnet_model_trainer
importlib.reload(resnet_model_trainer)

from resnet.resnet_model_trainer import ModelTrainer

# Training code
trainer = ModelTrainer(model, train_generator, val_generator, results_dir=DatasetConfig.PATHS['results'])
history = trainer.train()

In [ ]:
import model_evaluator
importlib.reload(model_evaluator)

from model_evaluator import ModelEvaluator

# Initialize evaluator and perform evaluation
print("Evaluating model performance...")
evaluator = ModelEvaluator(model, test_generator, DatasetConfig.PATHS['results'])
evaluator.evaluate(history)

## 5. DenseNet

In [ ]:
import densenet.densenet_model_builder as densenet_model_builder
importlib.reload(densenet_model_builder)

from densenet.densenet_model_builder import ModelBuilder

# Create model with input shape
model = ModelBuilder.create_model(input_shape=(224, 224, 3))

# Print model summary to verify architecture
model.summary()

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import densenet.densenet_model_trainer as densenet_model_trainer
importlib.reload(densenet_model_trainer)

from densenet.densenet_model_trainer import ModelTrainer

# Training code
trainer = ModelTrainer(model, train_generator, val_generator, results_dir=DatasetConfig.PATHS['results'])
history = trainer.train()

In [ ]:
import model_evaluator
importlib.reload(model_evaluator)

from model_evaluator import ModelEvaluator

# Initialize evaluator and perform evaluation
print("Evaluating model performance...")
evaluator = ModelEvaluator(model, test_generator, DatasetConfig.PATHS['results'])
evaluator.evaluate(history)

## 6. Evaluation on New Dataset

In [ ]:
import evaluation
importlib.reload(evaluation)
from evaluation import ModelPredictor

# Define paths
model_path = os.path.join('model', 'best_model.keras')
evaluation_dir = 'path/to/your/image/folder'

# Create predictor and run evaluation
predictor = ModelPredictor(
    model_path=model_path,
    evaluation_dir=evaluation_dir
)
predictor.predict_images()